In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import matrix_rank
from matplotlib.lines import Line2D
import sys
import os

if not os.environ.get('DISPLAY'):
    plt.switch_backend('Agg')

# LIF Network Class
class LIFNetwork:
    """Leaky Integrate-and-Fire (LIF) Spiking Neural Network (SNN) model.
    
    The connections are automatically generated as fully feedforward
    between adjacent layers.
    """
    def __init__(self, layers, tau=6.0, theta=0.4, reset=0.0, dt=0.001):
        self.layers = layers
        self.tau = tau
        self.theta = theta
        self.reset = reset
        self.dt = dt
        self.n_layers = len(layers)
        self.n = sum(layers) # Total number of neurons
        # Cumulative index of layers for easy slicing (e.g., [0, 1, 3, 4] for [1, 2, 1])
        self.layer_idx = np.cumsum([0] + layers)

        # Generic full feedforward connection generation (Post-synaptic index, Pre-synaptic index)
        self.connections = []
        for l in range(1, self.n_layers):
            # Iterate over neurons in the post-synaptic layer (l)
            for post in range(self.layer_idx[l], self.layer_idx[l+1]):
                # Iterate over neurons in the pre-synaptic layer (l-1)
                for pre in range(self.layer_idx[l-1], self.layer_idx[l]):
                    self.connections.append((post, pre))

    def simulate(self, x0, T, u):
        """Simulates the LIF network dynamics."""
        t = np.arange(0, T + self.dt, self.dt)
        x = np.zeros((len(t), self.n)) # Neuron membrane potentials
        x[0] = x0
        spikes = [[] for _ in range(self.n)]

        for i in range(len(t) - 1):
            # dx is initialized as a float array for the update at time i
            dx = -x[i].astype(float) / self.tau # Leak term
            
            # Synaptic input from connected neurons
            # Note: In this simple model, connection weight is implicitly 1/tau
            for post, pre in self.connections:
                dx[post] += x[i, pre] / self.tau
            
            # External input (u) to the input layer (layer 0)
            uval = np.array(u(t[i]), dtype=float) if callable(u) else np.array(u, dtype=float)
            
            # Ensure input dimensions match the input layer size
            input_size = self.layers[0]
            if len(uval) != input_size:
                raise ValueError(f"Input signal dimension ({len(uval)}) must match input layer size ({input_size}).")
            
            dx[:input_size] += uval[:input_size] / self.tau 
            
            # Euler integration step
            x[i + 1] = x[i] + dx * self.dt
            
            # Check for spikes and reset
            for idx in range(self.n):
                if x[i + 1, idx] >= self.theta:
                    spikes[idx].append(t[i + 1])
                    x[i + 1, idx] = self.reset
                    
        return t, x, spikes

    def controllability_observability(self):
        """Checks linear controllability and observability (approximate)."""
        # A is the matrix defining the internal dynamics (Leak + Neuron-to-Neuron interaction)
        A = -np.eye(self.n) / self.tau
        
        # Add connections based on the feedforward architecture
        for post, pre in self.connections:
            A[post, pre] += 1.0 / self.tau

        # B is the input matrix (Input only affects the first layer)
        B = np.zeros((self.n, self.layers[0]))
        for i in range(self.layers[0]):
            B[i, i] = 1.0 / self.tau
            
        # C is the output matrix (measuring all states)
        C = np.eye(self.n)
        
        # Controllability Matrix (CM)
        CM = B.copy()
        for i in range(1, self.n):
            # CM = [B, A*B, A^2*B, ...]
            # Using matrix multiplication with A and the last block of B columns
            CM = np.hstack((CM, A @ CM[:, -B.shape[1]:])) 
            
        ctrl = matrix_rank(CM) == self.n
        
        # Observability Matrix (OM)
        OM = C.copy()
        for i in range(1, self.n):
            # OM = [C^T, (C*A)^T, (C*A^2)^T, ...]^T
            OM = np.vstack((OM, C @ np.linalg.matrix_power(A, i)))
            
        obs = matrix_rank(OM) == self.n
        
        return ctrl, obs

# Draw architecture diagram

def draw_architecture(ax, layers, connections, title):
    """Draws the SNN architecture diagram."""
    
    global_to_pos = {} 
    global_idx = 0
    
    x_positions = np.linspace(0.1, 0.9, len(layers))

    colors = {
        0: "#3498DB", # Input (Vibrant Blue)
        -1: "#2ECC71", # Output (Vibrant Green)
        "hidden": "#E74C3C" # Hidden (Vibrant Red/Rose)
    }

    # 1. Create nodes and map their global positions
    for layer_index, count in enumerate(layers):
        if layer_index == 0:
            c = colors[0] 
        elif layer_index == len(layers) - 1:
            c = colors[-1] 
        else:
            c = colors["hidden"]

        # Adjust y-positions to center the layers better
        y_positions = np.linspace(0.9, 0.1, count)
        xs = np.full(count, x_positions[layer_index])

        # Draw the nodes
        ax.scatter(xs, y_positions, s=500, color=c, edgecolor='black', zorder=3)
        
        # Store global position mapping
        for i, (x, y) in enumerate(zip(xs, y_positions)):
            global_to_pos[global_idx] = (x, y)
            global_idx += 1

    # 2. Draw connections
    for post, pre in connections:
        if pre in global_to_pos and post in global_to_pos:
            xa, ya = global_to_pos[pre] 
            xb, yb = global_to_pos[post]
            # Draw line from pre to post
            ax.plot([xa, xb], [ya, yb], color='gray', linewidth=0.8, zorder=2)

    # 3. Title and Axis
    ax.set_title(title, fontsize=13)
    ax.axis('off')

    # 4. Add color legend
    legend_labels = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor=colors[0], markersize=10, label="Entrada (Input)"),
        Line2D([0], [0], marker='o', color='w', markerfacecolor=colors["hidden"], markersize=10, label="Capa Oculta (Hidden)"),
        Line2D([0], [0], marker='o', color='w', markerfacecolor=colors[-1], markersize=10, label="Salida (Output)")
    ]
    ax.legend(handles=legend_labels, loc='upper left', fontsize=10)

# Plot experiment 

def plot_experiment(sys, x0, T, input_signal, title):
    """Runs simulation and generates 5 independent figures."""
    print(f"Running simulation for {title}...")
    t, x, spikes = sys.simulate(x0, T, input_signal)
    ctrl, obs = sys.controllability_observability()
    print(f"Linear Controllability: {ctrl}, Observability: {obs}")

    # --- 1) Arquitectura (Architecture) ---
    fig1, ax1 = plt.subplots(figsize=(10, 6))
    draw_architecture(ax1, sys.layers, sys.connections, f"{title} - Arquitectura (Estructura: {sys.layers})")
    fig1.tight_layout()
    plt.show()

    # --- 2) Entrada (Input) ---
    fig2, ax2 = plt.subplots(figsize=(10, 3))
    uvals = np.array([input_signal(ti) if callable(input_signal) else input_signal for ti in t])
    
    input_dim = sys.layers[0]
    #flat case
    if uvals.ndim == 1 and input_dim > 1:
        uvals = uvals.reshape(-1, input_dim)
    elif uvals.ndim == 1 and input_dim == 1:
        # Single input, reshape to (T, 1) if necessary
        uvals = uvals.reshape(-1, 1)

    for i in range(input_dim):
        ax2.plot(t, uvals[:, i], label=f"Input {i+1}", linestyle='--' if input_dim > 1 else '-')
        
    ax2.set_ylabel("Entrada")
    ax2.grid(True)
    ax2.legend() 
    ax2.set_title(f"Entrada | Linealmente Controlable: {ctrl}, Linealmente Observable: {obs}")
    ax2.set_xlabel("Tiempo (s)")
    fig2.tight_layout()
    plt.show()

    # --- 3) Potenciales de neuronas ocultas ---
    fig3, ax3 = plt.subplots(figsize=(10, 4))
    hidden_start = sys.layers[0]
    hidden_end = sys.n - sys.layers[-1]
    
    if hidden_end > hidden_start: 
        for idx in range(hidden_start, hidden_end):
            # Using high-contrast color palette
            color_index = idx - hidden_start
            color = plt.cm.get_cmap('Set1')(color_index % 9)
            ax3.plot(t, x[:, idx], label=f"Oculta {idx - hidden_start + 1}", color=color)
        
        ax3.axhline(sys.theta, color='gray', linestyle=':', label='Umbral')

    else: 
        ax3.text(0.5, 0.5, "No hay capa oculta para mostrar", ha='center', va='center')
        
    ax3.set_ylabel("Potencial Oculto")
    ax3.set_title("Potenciales de la capa oculta")
    ax3.grid(True)
    ax3.legend()
    ax3.set_xlabel("Tiempo (s)")
    fig3.tight_layout()
    plt.show()

    # --- 4) Membrana de salida + spikes (Output membrane + spikes) ---
    fig4, ax4 = plt.subplots(figsize=(10, 4))
    output_idx = sys.n - 1 # Assuming single output neuron
    ax4.plot(t, x[:, output_idx], label="Membrana salida", color='#2ECC71', linewidth=2)
    ax4.axhline(sys.theta, color='#E74C3C', linestyle='--', label='Umbral')
    
    # Plot spikes slightly above the threshold line
    ax4.eventplot(spikes[output_idx], lineoffsets=sys.theta + 0.05, colors='black', linelength=0.1)
    ax4.set_ylabel("Potencial Salida")
    ax4.set_xlabel("Tiempo (s)")
    ax4.legend()
    ax4.grid(True)
    ax4.set_title("Membrana de salida y Umbral")
    fig4.tight_layout()
    plt.show()

    # --- 5) Spikes de salida solamente (Output spikes only) ---
    fig5, ax5 = plt.subplots(figsize=(10, 2))
    ax5.eventplot(spikes[output_idx], lineoffsets=1, colors='k')
    ax5.set_ylabel("Spike")
    ax5.set_xlabel("Tiempo (s)")
    ax5.set_title("Spikes de salida")
    ax5.set_yticks([])
    ax5.grid(True)
    fig5.tight_layout()
    plt.show()

# ============================================================
# Run single analogous experiment (2 -> 3 -> 1)
# ============================================================

def main():
    # Architectures commonly used in SNN control literature, prioritizing high controllability (large L0/n)
    architectures_to_test = [
        [3, 1, 1], # n=5, L0=3 (Best candidate for controllability)
        [2, 2, 1], # n=5, L0=2 
        [2, 3, 1], # n=6, L0=2
        [3, 2, 1], # n=6, L0=3 
        [4, 1, 1], # n=6, L0=4 (High controllability)
        [1, 1, 1]  # n=3, L0=1 (Simple minimum structure)
    ]
    
    T = 10.0  # Simulation time (seconds)
    found_controllable = False

    print("--- Searching for a Controllable LIF Architecture for Shooting Method Analogy ---")
    
    for LAYERS in architectures_to_test:
        sys_search = LIFNetwork(LAYERS, tau=10.0, theta=1.0, reset=0.0, dt=0.001)
        # Check controllability without running the full simulation
        ctrl, obs = sys_search.controllability_observability()
        
        print(f"Checking architecture {LAYERS} (n={sys_search.n}, L0={LAYERS[0]}): Controllable={ctrl}")
        
        if ctrl:
            print(f"\n--- Found Controllable Architecture: {LAYERS} ---")
            
            # Setup inputs for simulation
            L0 = LAYERS[0]
            x0 = np.zeros(sys_search.n)
            # Input must match the found input size L0. Set constant input of 1.5 for all L0 inputs.
            u_input = lambda t: [1.5] * L0
            
            # Run the full simulation and plotting for the first controllable system found
            plot_experiment(sys_search, x0, T, u_input, 
                            f"Controllable SNN Model Found: {LAYERS[0]}->{LAYERS[1]}->{LAYERS[2]}")
            
            found_controllable = True
            break # Stop after finding the first one
            
    if not found_controllable:
        print("\n--- Search completed. No controllable architecture found in the test list. ---")

if __name__ == "__main__":
    main()

--- Searching for a Controllable LIF Architecture for Shooting Method Analogy ---
Checking architecture [3, 1, 1] (n=5, L0=3): Controllable=True

--- Found Controllable Architecture: [3, 1, 1] ---
Running simulation for Controllable SNN Model Found: 3->1->1...
Linear Controllability: True, Observability: True


C:\Users\tito\AppData\Local\Temp\ipykernel_62784\4188111312.py:181: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\tito\AppData\Local\Temp\ipykernel_62784\4188111312.py:204: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\tito\AppData\Local\Temp\ipykernel_62784\4188111312.py:215: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  color = plt.cm.get_cmap('Set1')(color_index % 9)
C:\Users\tito\AppData\Local\Temp\ipykernel_62784\4188111312.py:229: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\tito\AppData\Local\Temp\ipykernel_62784\4188111312.py:245: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\tito\AppData\Local\Temp\ipyker

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import matrix_rank
from matplotlib.lines import Line2D

# LIF Network Class
class LIFNetwork:
    """Leaky Integrate-and-Fire (LIF) Spiking Neural Network (SNN) model.
    
    The connections are automatically generated as fully feedforward
    between adjacent layers.
    """
    def __init__(self, layers, tau=6.0, theta=0.4, reset=0.0, dt=0.001):
        self.layers = layers
        self.tau = tau
        self.theta = theta
        self.reset = reset
        self.dt = dt
        self.n_layers = len(layers)
        self.n = sum(layers) # Total number of neurons
        # Cumulative index of layers for easy slicing (e.g., [0, 1, 3, 4] for [1, 2, 1])
        self.layer_idx = np.cumsum([0] + layers)

        # Generic full feedforward connection generation (Post-synaptic index, Pre-synaptic index)
        self.connections = []
        for l in range(1, self.n_layers):
            # Iterate over neurons in the post-synaptic layer (l)
            for post in range(self.layer_idx[l], self.layer_idx[l+1]):
                # Iterate over neurons in the pre-synaptic layer (l-1)
                for pre in range(self.layer_idx[l-1], self.layer_idx[l]):
                    self.connections.append((post, pre))

    def simulate(self, x0, T, u):
        """Simulates the LIF network dynamics."""
        t = np.arange(0, T + self.dt, self.dt)
        x = np.zeros((len(t), self.n)) # Neuron membrane potentials
        x[0] = x0
        spikes = [[] for _ in range(self.n)]

        for i in range(len(t) - 1):
            # dx is initialized as a float array for the update at time i
            dx = -x[i].astype(float) / self.tau # Leak term
            
            # Synaptic input from connected neurons
            # Note: In this simple model, connection weight is implicitly 1/tau
            for post, pre in self.connections:
                dx[post] += x[i, pre] / self.tau
            
            # External input (u) to the input layer (layer 0)
            uval = np.array(u(t[i]), dtype=float) if callable(u) else np.array(u, dtype=float)
            
            # Ensure input dimensions match the input layer size
            input_size = self.layers[0]
            if len(uval) != input_size:
                raise ValueError(f"Input signal dimension ({len(uval)}) must match input layer size ({input_size}).")
            
            dx[:input_size] += uval[:input_size] / self.tau 
            
            # Euler integration step
            x[i + 1] = x[i] + dx * self.dt
            
            # Check for spikes and reset
            for idx in range(self.n):
                if x[i + 1, idx] >= self.theta:
                    spikes[idx].append(t[i + 1])
                    x[i + 1, idx] = self.reset
                    
        return t, x, spikes

    def controllability_observability(self):
        """Checks linear controllability and observability (approximate)."""
        # A is the matrix defining the internal dynamics (Leak + Neuron-to-Neuron interaction)
        A = -np.eye(self.n) / self.tau
        
        # Add connections based on the feedforward architecture
        for post, pre in self.connections:
            A[post, pre] += 1.0 / self.tau

        # B is the input matrix (Input only affects the first layer)
        B = np.zeros((self.n, self.layers[0]))
        for i in range(self.layers[0]):
            B[i, i] = 1.0 / self.tau
            
        # C is the output matrix (measuring all states)
        C = np.eye(self.n)
        
        # Controllability Matrix (CM)
        CM = B.copy()
        for i in range(1, self.n):
            # CM = [B, A*B, A^2*B, ...]
            # Using matrix multiplication with A and the last block of B columns
            CM = np.hstack((CM, A @ CM[:, -B.shape[1]:])) 
            
        ctrl = matrix_rank(CM) == self.n
        
        # Observability Matrix (OM)
        OM = C.copy()
        for i in range(1, self.n):
            # OM = [C^T, (C*A)^T, (C*A^2)^T, ...]^T
            OM = np.vstack((OM, C @ np.linalg.matrix_power(A, i)))
            
        obs = matrix_rank(OM) == self.n
        
        return ctrl, obs

# Draw architecture diagram

def draw_architecture(ax, layers, connections, title):
    """Draws the SNN architecture diagram."""
    
    global_to_pos = {} 
    global_idx = 0
    
    x_positions = np.linspace(0.1, 0.9, len(layers))

    colors = {
        0: "#3498DB", # Input (Vibrant Blue)
        -1: "#2ECC71", # Output (Vibrant Green)
        "hidden": "#E74C3C" # Hidden (Vibrant Red/Rose)
    }

    # 1. Create nodes and map their global positions
    for layer_index, count in enumerate(layers):
        if layer_index == 0:
            c = colors[0] 
        elif layer_index == len(layers) - 1:
            c = colors[-1] 
        else:
            c = colors["hidden"]

        # Adjust y-positions to center the layers better
        y_positions = np.linspace(0.9, 0.1, count)
        xs = np.full(count, x_positions[layer_index])

        # Draw the nodes
        ax.scatter(xs, y_positions, s=500, color=c, edgecolor='black', zorder=3)
        
        # Store global position mapping
        for i, (x, y) in enumerate(zip(xs, y_positions)):
            global_to_pos[global_idx] = (x, y)
            global_idx += 1

    # 2. Draw connections
    for post, pre in connections:
        if pre in global_to_pos and post in global_to_pos:
            xa, ya = global_to_pos[pre] 
            xb, yb = global_to_pos[post]
            # Draw line from pre to post
            ax.plot([xa, xb], [ya, yb], color='gray', linewidth=0.8, zorder=2)

    # 3. Title and Axis
    ax.set_title(title, fontsize=13)
    ax.axis('off')

    # 4. Add color legend
    legend_labels = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor=colors[0], markersize=10, label="Entrada (Input)"),
        Line2D([0], [0], marker='o', color='w', markerfacecolor=colors["hidden"], markersize=10, label="Capa Oculta (Hidden)"),
        Line2D([0], [0], marker='o', color='w', markerfacecolor=colors[-1], markersize=10, label="Salida (Output)")
    ]
    ax.legend(handles=legend_labels, loc='upper left', fontsize=10)

# Plot experiment and save figures

def generate_figures(sys, x0, T, input_signal, title):
    """Runs simulation and generates 5 independent figures, saving them."""
    print(f"Running simulation for {title}...")
    t, x, spikes = sys.simulate(x0, T, input_signal)
    ctrl, obs = sys.controllability_observability()
    print(f"Linear Controllability: {ctrl}, Observability: {obs}")

    # --- 1) Arquitectura (Architecture) ---
    fig1, ax1 = plt.subplots(figsize=(10, 6))
    draw_architecture(ax1, sys.layers, sys.connections, f"{title} - Arquitectura (Estructura: {sys.layers})")
    fig1.tight_layout()
    plt.savefig('controllability_architecture.png', dpi=300, bbox_inches='tight')
    print("Saved: controllability_architecture.png")
    plt.close(fig1)

    # --- 2) Entrada (Input) ---
    fig2, ax2 = plt.subplots(figsize=(10, 3))
    uvals = np.array([input_signal(ti) if callable(input_signal) else input_signal for ti in t])
    
    input_dim = sys.layers[0]
    #flat case
    if uvals.ndim == 1 and input_dim > 1:
        uvals = uvals.reshape(-1, input_dim)
    elif uvals.ndim == 1 and input_dim == 1:
        # Single input, reshape to (T, 1) if necessary
        uvals = uvals.reshape(-1, 1)

    for i in range(input_dim):
        ax2.plot(t, uvals[:, i], label=f"Input {i+1}", linestyle='--' if input_dim > 1 else '-')
        
    ax2.set_ylabel("Entrada")
    ax2.grid(True)
    ax2.legend() 
    ax2.set_title(f"Entrada | Linealmente Controlable: {ctrl}, Linealmente Observable: {obs}")
    ax2.set_xlabel("Tiempo (s)")
    fig2.tight_layout()
    plt.savefig('controllability_input.png', dpi=300, bbox_inches='tight')
    print("Saved: controllability_input.png")
    plt.close(fig2)

    # --- 3) Potenciales de neuronas ocultas ---
    fig3, ax3 = plt.subplots(figsize=(10, 4))
    hidden_start = sys.layers[0]
    hidden_end = sys.n - sys.layers[-1]
    
    if hidden_end > hidden_start: 
        for idx in range(hidden_start, hidden_end):
            # Using high-contrast color palette
            color_index = idx - hidden_start
            color = plt.cm.get_cmap('Set1')(color_index % 9)
            ax3.plot(t, x[:, idx], label=f"Oculta {idx - hidden_start + 1}", color=color)
        
        ax3.axhline(sys.theta, color='gray', linestyle=':', label='Umbral')

    else: 
        ax3.text(0.5, 0.5, "No hay capa oculta para mostrar", ha='center', va='center')
        
    ax3.set_ylabel("Potencial Oculto")
    ax3.set_title("Potenciales de la capa oculta")
    ax3.grid(True)
    ax3.legend()
    ax3.set_xlabel("Tiempo (s)")
    fig3.tight_layout()
    plt.savefig('controllability_hidden.png', dpi=300, bbox_inches='tight')
    print("Saved: controllability_hidden.png")
    plt.close(fig3)

    # --- 4) Membrana de salida + spikes (Output membrane + spikes) ---
    fig4, ax4 = plt.subplots(figsize=(10, 4))
    output_idx = sys.n - 1 # Assuming single output neuron
    ax4.plot(t, x[:, output_idx], label="Membrana salida", color='#2ECC71', linewidth=2)
    ax4.axhline(sys.theta, color='#E74C3C', linestyle='--', label='Umbral')
    
    # Plot spikes slightly above the threshold line
    ax4.eventplot(spikes[output_idx], lineoffsets=sys.theta + 0.05, colors='black', linelength=0.1)
    ax4.set_ylabel("Potencial Salida")
    ax4.set_xlabel("Tiempo (s)")
    ax4.legend()
    ax4.grid(True)
    ax4.set_title("Membrana de salida y Umbral")
    fig4.tight_layout()
    plt.savefig('controllability_output_membrane.png', dpi=300, bbox_inches='tight')
    print("Saved: controllability_output_membrane.png")
    plt.close(fig4)

    # --- 5) Spikes de salida solamente (Output spikes only) ---
    fig5, ax5 = plt.subplots(figsize=(10, 2))
    ax5.eventplot(spikes[output_idx], lineoffsets=1, colors='k')
    ax5.set_ylabel("Spike")
    ax5.set_xlabel("Tiempo (s)")
    ax5.set_title("Spikes de salida")
    ax5.set_yticks([])
    ax5.grid(True)
    fig5.tight_layout()
    plt.savefig('controllability_spikes.png', dpi=300, bbox_inches='tight')
    print("Saved: controllability_spikes.png")
    plt.close(fig5)

def main():
    # Use the architecture that was found to be controllable: [3, 1, 1]
    LAYERS = [3, 1, 1]
    
    print("=== Generating Controllability Figures for [3, 1, 1] Architecture ===")
    
    sys = LIFNetwork(LAYERS, tau=10.0, theta=1.0, reset=0.0, dt=0.001)
    ctrl, obs = sys.controllability_observability()
    
    print(f"Architecture {LAYERS} (n={sys.n}, L0={LAYERS[0]}): Controllable={ctrl}, Observable={obs}")
    
    if ctrl:
        T = 10.0  # Simulation time (seconds)
        x0 = np.zeros(sys.n)
        L0 = LAYERS[0]
        # Input must match the input size L0. Set constant input of 1.5 for all L0 inputs.
        u_input = lambda t: [1.5] * L0
        
        # Generate all figures
        generate_figures(sys, x0, T, u_input, 
                        f"Red Neuronal por Impulsos Controlable: {LAYERS[0]}→{LAYERS[1]}→{LAYERS[2]}")
        
        print("\n=== All figures generated successfully ===")
    else:
        print("ERROR: Architecture is not controllable!")

if __name__ == "__main__":
    main()

=== Generating Controllability Figures for [3, 1, 1] Architecture ===
Architecture [3, 1, 1] (n=5, L0=3): Controllable=True, Observable=True
Running simulation for Red Neuronal por Impulsos Controlable: 3→1→1...
Linear Controllability: True, Observability: True
Saved: controllability_architecture.png
Saved: controllability_input.png


C:\Users\tito\AppData\Local\Temp\ipykernel_62784\179263517.py:214: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  color = plt.cm.get_cmap('Set1')(color_index % 9)


Saved: controllability_hidden.png
Saved: controllability_output_membrane.png
Saved: controllability_spikes.png

=== All figures generated successfully ===


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c7d4307c-0ebe-4a0e-91ec-84ab60ddfdb3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>